In [2]:
!pip install webrtcvad

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for webrtcvad
Failed to build webrtcvad


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [10 lines of output]
      C:\Python\Lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312
      copying webrtcvad.py -> build\lib.win-amd64-cpython-312
      running build_ext
      building '_webrtcvad' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for webrtcvad
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (webrtcvad)


In [4]:
!pip install pyAudioAnalysis

     ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
     - -------------------------------------- 1.0/41.3 MB 5.6 MB/s eta 0:00:08
     --- ------------------------------------ 3.1/41.3 MB 7.7 MB/s eta 0:00:05
     ----- ---------------------------------- 6.0/41.3 MB 10.3 MB/s eta 0:00:04
     ------- -------------------------------- 7.9/41.3 MB 9.7 MB/s eta 0:00:04
     -------- ------------------------------- 9.2/41.3 MB 8.9 MB/s eta 0:00:04
     ----------- ---------------------------- 11.5/41.3 MB 9.3 MB/s eta 0:00:04
     -------------- ------------------------ 14.9/41.3 MB 10.2 MB/s eta 0:00:03
     ----------------- --------------------- 18.9/41.3 MB 11.2 MB/s eta 0:00:02
     --------------------- ----------------- 22.3/41.3 MB 11.8 MB/s eta 0:00:02
     ----------------------- --------------- 24.4/41.3 MB 11.7 MB/s eta 0:00:02
     ------------------------- ------------- 26.5/41.3 MB 11.6 MB/s eta 0:00:02
     --------------------------- ----------- 29.4/41

In [12]:
!pip install plotly

   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   -- ------------------------------------- 1.0/19.1 MB 6.3 MB/s eta 0:00:03
   ---- ----------------------------------- 2.1/19.1 MB 5.9 MB/s eta 0:00:03
   -------- ------------------------------- 3.9/19.1 MB 7.1 MB/s eta 0:00:03
   ------------- -------------------------- 6.3/19.1 MB 8.2 MB/s eta 0:00:02
   ------------------- -------------------- 9.2/19.1 MB 9.5 MB/s eta 0:00:02
   --------------------- ------------------ 10.2/19.1 MB 8.6 MB/s eta 0:00:02
   ------------------------- -------------- 12.1/19.1 MB 8.7 MB/s eta 0:00:01
   ------------------------------ --------- 14.4/19.1 MB 9.0 MB/s eta 0:00:01
   ----------------------------------- ---- 17.0/19.1 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------  18.9/19.1 MB 9.7 MB/s eta 0:00:01
   ---------------------------------------- 19.1/19.1 MB 9.3 MB/s eta 0:00:00


In [13]:
from pyAudioAnalysis import audioSegmentation as aS

# Set the file path
file_path = 'D:\\telelyzerv2\\experiment\\exp_data\\example of audio dropouts.mp3'  # Replace with your file path

# Perform silence removal to identify speech segments
segments, _ = aS.silence_removal(file_path, 16000, 0.020, 0.020, smooth_window=1.0, weight=0.5, plot=True)

# Print detected speech segments
print("Detected Speech Segments:")
for i, (start, end) in enumerate(segments, 1):
    print(f"Segment {i}: Start = {start:.2f}s, End = {end:.2f}s, Duration = {end - start:.2f}s")


AttributeError: 'str' object has no attribute 'ndim'

In [23]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa

# Load the Wav2Vec2 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Load audio
file_path = 'D:\\telelyzerv2\\experiment\\exp_data\\example of audio dropouts.mp3'
audio, sr = librosa.load(file_path, sr=16000)

# Segment the audio into frames and check if there is transcribed text
frame_duration = int(1 * sr)  # Half-second frames
speech_segments = []
non_speech_segments = []
silence_threshold = 10.0  # 1 second threshold for long pauses
pause_segments = []
current_pause_start = None

for i in range(0, len(audio), frame_duration):
    frame = audio[i:i + frame_duration]
    input_values = processor(frame, sampling_rate=sr, return_tensors="pt").input_values
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])

    # Check if transcription has content to determine speech or non-speech
    start_time = i / sr
    end_time = (i + frame_duration) / sr

    print(transcription)
    
    if transcription.strip():  # Speech detected
        speech_segments.append((start_time, end_time))
        # If a pause was being tracked, finalize it
        if current_pause_start is not None:
            pause_duration = end_time - current_pause_start
            if pause_duration >= silence_threshold:
                pause_segments.append((current_pause_start, end_time))
            current_pause_start = None
    else:  # Non-speech detected
        non_speech_segments.append((start_time, end_time))
        # Start tracking pause if not already tracking
        if current_pause_start is None:
            current_pause_start = start_time

# Finalize any remaining pause if the audio ends with silence
if current_pause_start is not None:
    pause_duration = (len(audio) / sr) - current_pause_start
    if pause_duration >= silence_threshold:
        pause_segments.append((current_pause_start, len(audio) / sr))

# # Print detected speech and non-speech segments
# print("Detected Speech Segments:")
# for start, end in speech_segments:
#     print(f"Start = {start:.2f}s, End = {end:.2f}s, Duration = {end - start:.2f}s")

# print("\nDetected Non-Speech Segments:")
# for start, end in non_speech_segments:
#     print(f"Start = {start:.2f}s, End = {end:.2f}s, Duration = {end - start:.2f}s")

print("\nDetected Long Pauses:")
for start, end in pause_segments:
    print(f"Start = {start:.2f}s, End = {end:.2f}s, Duration = {end - start:.2f}s")


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Python\Lib\site-packages\pyAudioAnalysis\..\transformers\models\wav2vec2\feature_extraction_wav2vec2.py:98: RuntimeWarning: Mean of empty slice.
  normed_input_values = [(x - x.mean()) / np.sqrt(x.var() + 1e-7) for x in input_values]
c:\Python\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
C:\Python\Lib\site-packages\pyAudioAnalysis\..\transformers\models\wav2vec2\feature_extraction_wav2vec2.py:98: RuntimeWarning: Degrees of freedom <= 0 for slice
  normed_input_values = [(x - x.mean()) / np.sqrt(x.var() + 1e-7) for x in input_values]
c:\Python\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value enco

RuntimeError: Calculated padded input size per channel: (0). Kernel size: (10). Kernel size can't be greater than actual input size